# Punto 4

## Librerías Utilizadas

In [16]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from hyperopt import hp, fmin, tpe, Trials, space_eval, STATUS_OK
import warnings
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
warnings.filterwarnings("ignore")

Iniciamos cargando el conjunto de datos

In [9]:
wind = pd.read_csv("https://raw.githubusercontent.com/lihkir/Data/main/wind_speed/data_treino_dv_df_2000_2010.csv")

Renombranos las columnas

In [10]:
columnas = {'HORA (UTC)': 'hora', 'VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))': 'direccion_viento', 
            'VENTO, VELOCIDADE HORARIA (m/s)': 'velocidad_viento', 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'humedad_max',
            'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'humedad_min', 'TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C)': 'temperatura_max',
            'TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C)': 'temperatura_min', 'UMIDADE RELATIVA DO AR, HORARIA (%)': 'humedad_horaria',
            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pres_atmosferica', 'PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)': 'precipitacion_hora',
            'VENTO, RAJADA MAXIMA (m/s)': 'rafaga_max', 'PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'pres_atmosferica_max', 'PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'pres_atmosferica_min'}
wind = wind.rename(columns=columnas)

Observamos la naturaleza de la cada columna y comprobamos que no hayan nulos.

In [11]:
wind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87693 entries, 0 to 87692
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   hora                  87693 non-null  object 
 1   direccion_viento      87693 non-null  float64
 2   velocidad_viento      87693 non-null  float64
 3   humedad_max           87693 non-null  float64
 4   humedad_min           87693 non-null  float64
 5   temperatura_max       87693 non-null  float64
 6   temperatura_min       87693 non-null  float64
 7   humedad_horaria       87693 non-null  float64
 8   pres_atmosferica      87693 non-null  float64
 9   precipitacion_hora    87693 non-null  float64
 10  rafaga_max            87693 non-null  float64
 11  pres_atmosferica_max  87693 non-null  float64
 12  pres_atmosferica_min  87693 non-null  float64
dtypes: float64(12), object(1)
memory usage: 8.7+ MB


In [12]:
wind.head()

,hora,direccion_viento,velocidad_viento,humedad_max,humedad_min,temperatura_max,temperatura_min,humedad_horaria,pres_atmosferica,precipitacion_hora,rafaga_max,pres_atmosferica_max,pres_atmosferica_min
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2
2,14:00,0.891007,2.0,56.0,50.0,25.5,24.3,51.0,888.1,0.0,4.9,888.4,888.1
3,15:00,0.848048,2.5,52.0,44.0,27.4,25.0,44.0,887.4,0.0,5.8,888.1,887.4
4,16:00,0.224951,2.4,50.0,43.0,27.1,25.5,46.0,886.5,0.0,5.8,887.4,886.5


Observamos las primeras filas del Dataset.

Dividimos los datos en las variables explicativas y la variable respuesta.

In [13]:
X = wind.drop(columns=['velocidad_viento', 'hora'])
y = wind['velocidad_viento']

Diccionario de los modelos de regresión que serán implementados.

In [14]:
model_dict = {
    'SVM': Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=3)),
        ('svm', SVR(kernel='rbf'))
    ]),
    'Random Forest': Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=3)),
        ('rf', RandomForestRegressor(n_jobs=-1))
    ])
}

Aqui se observara algunos pliegues para cada ventana T, para demostrar que lo hace correctamente, se mostraran 5 por convenencia, pero ya al entrenar el modelo si se usaran todas

In [15]:
# Establecemos las ventanas temporales que queremos evaluar
time_windows = [7, 14, 21, 28]

# Lista para recopilar los resultados
# Verificar los índices
for T in time_windows:
    count = 0  # Inicializamos el contador para cada ventana
    for start in range(0, len(X) - T):
        if count >= 5:  # Verificamos si ya hemos impreso 5 ventanas
            break
        end = start + T
        
        print(f"Ventana T={T}:")
        print(f"Índices de entrenamiento: {list(range(start, end))}")
        print(f"Índices de prueba: {list(range(end, end + 1))}")
        print("---")
        
        count += 1  # Incrementamos el contador

Ventana T=7:
Índices de entrenamiento: [0, 1, 2, 3, 4, 5, 6]
Índices de prueba: [7]
---
Ventana T=7:
Índices de entrenamiento: [1, 2, 3, 4, 5, 6, 7]
Índices de prueba: [8]
---
Ventana T=7:
Índices de entrenamiento: [2, 3, 4, 5, 6, 7, 8]
Índices de prueba: [9]
---
Ventana T=7:
Índices de entrenamiento: [3, 4, 5, 6, 7, 8, 9]
Índices de prueba: [10]
---
Ventana T=7:
Índices de entrenamiento: [4, 5, 6, 7, 8, 9, 10]
Índices de prueba: [11]
---
Ventana T=14:
Índices de entrenamiento: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Índices de prueba: [14]
---
Ventana T=14:
Índices de entrenamiento: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Índices de prueba: [15]
---
Ventana T=14:
Índices de entrenamiento: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Índices de prueba: [16]
---
Ventana T=14:
Índices de entrenamiento: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
Índices de prueba: [17]
---
Ventana T=14:
Índices de entrenamiento: [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


Entrenamos cada modelo y obtenemos sus métricas para cada ventana de predicción: T= 7, 14, 21, 28


In [24]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / (np.mean(actual))
    return res

def MAPE(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

# Establecemos las ventanas temporales que queremos evaluar
time_windows = [7, 14, 21, 28]

# Lista para recopilar los resultados
all_results = []

# Iterar sobre cada ventana temporal
for T in time_windows:
    for start in range(0, len(X) - T):
        end = start + T
        
        X_train, y_train = X.iloc[start:end], y.iloc[start:end]
        X_test, y_test = X.iloc[end:end + 1], y.iloc[end:end + 1]
        
        # Verificar que no hay valores NaN o infinitos en los conjuntos de entrenamiento y prueba
        if X_train.isna().any().any() or y_train.isna().any() or X_test.isna().any().any() or y_test.isna().any():
            continue
        if np.isinf(X_train.values).any() or np.isinf(y_train.values).any() or np.isinf(X_test.values).any() or np.isinf(y_test.values).any():
            continue
        
        # Entrenar y evaluar cada modelo
        for name, model in model_dict.items():
            start_time = time.time()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            end_time = time.time()
            
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            elapsed_time = end_time - start_time
            
            all_results.append({
                'Modelo': name,
                'RMSE': rmse,
                'Tiempo': elapsed_time
            })

results_df = pd.DataFrame(all_results)
average_results = results_df.groupby('Modelo').mean().reset_index()
best_model = average_results.loc[average_results['RMSE'].idxmin()] 

print(average_results)
print(f"El mejor modelo es: {best_model['Modelo']} con un RMSE de {best_model['RMSE']}")

 El modelo de Random Forest tiene el RMSE más bajo (0.693801) y SVM con un RMSE de (0.720594) lo que indica que el modelo con el mejor rendimiento general presentado es el Random Forest para estos datos. 

Ahora realizamos la optimización bayesiana para obtener los mejores hiperparámetros para Radom Forest.

In [15]:
# Definición del espacio de búsqueda
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),  
    'max_depth': hp.quniform('max_depth', 10, 30, 1),           
    'min_samples_split': hp.quniform('min_samples_split', 2, 20, 1),  
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 20, 1),    
    'max_features': hp.uniform('max_features', 0.1, 0.9),             
    'bootstrap': hp.choice('bootstrap', [True, False])           
}

# Creación de la función objetivo
def hyperopt_objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])
    clf = RandomForestRegressor(**params, n_jobs=-1)
    score = -cross_val_score(clf, X, y, cv=5, scoring='neg_mean_squared_error').mean()
    return {'loss': score, 'status': STATUS_OK}

# Ejecución de la optimización bayesiana
trials=Trials()
best_params = fmin(
    fn=hyperopt_objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials 
)

# Calcula los hiperparámetros reales a partir de los índices
best_params = space_eval(space, best_params)

print(f"Mejores hiperparámetros encontrados: {best_params}")

100%|██████████| 100/100 [1:02:19<00:00, 37.40s/trial, best loss: 0.32683632778260685]
Mejores hiperparámetros encontrados: {'bootstrap': True, 'max_depth': 30.0, 'max_features': 0.5533599223455185, 'min_samples_leaf': 10.0, 'min_samples_split': 15.0, 'n_estimators': 661.0}


Los mejores hiperparámetros obtenidos por la optimización  bayesiana son:
- 'bootstrap': True

- 'max_depth': 30.0

- 'max_features': 0.5533599223455185

- 'min_samples_leaf': 10.0

- 'min_samples_split': 15.0

- 'n_estimators': 661.0

Ahora, usando los mejores hiperparámetros que obtuvimos, volveremos a ajustar el modelo y encontraremos la mejor ventana de predicción.

In [28]:
best_params = {
    'bootstrap': True,
    'max_depth': int(30.0),
    'max_features': 0.5533599223455185, 
    'min_samples_leaf': int(10.0),
    'min_samples_split': int(15.0),
    'n_estimators': int(661.0)
}

best_rf = RandomForestRegressor(**best_params)

time_windows = [7, 14, 21, 28]
results_by_window = []

for T in time_windows:
    rmse_list = []

    for start in range(0, len(X) - T):
        end = start + T
        
        X_train, y_train = X.iloc[start:end], y.iloc[start:end]
        X_test, y_test = X.iloc[end:end + 1], y.iloc[end:end + 1]
        
        best_rf.fit(X_train, y_train)
        y_pred = best_rf.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        rmse_list.append(rmse)

    results_by_window.append({
        'Ventana Temporal': T,
        'RMSE Mean': np.mean(rmse_list),
        'RMSE Std': np.std(rmse_list)
    })

final_results_df = pd.DataFrame(results_by_window)
final_results_df.sort_values('RMSE Mean', inplace=True)
print(final_results_df)

   Ventana Temporal  RMSE Mean  RMSE Std
3                28   0.860146  0.661801
0                 7   0.870943  0.692047
2                21   0.963227  0.708119
1                14   0.994732  0.737538


In [30]:
best_window = final_results_df.iloc[0]
print(f"Ventana temporal con el mejor rendimiento para Random Forest: T = {best_window['Ventana Temporal']} días con un RMSE de {best_window['RMSE Mean']}")

Ventana temporal con el mejor rendimiento para Random Forest: T = 28.0 días con un RMSE de 0.8601462482072594


Teniendo en cuenta las métricas podemos concluir que la mejor ventana de predicción es 28 con un RMSE de (0.86)